# Import

In [ ]:
import pandas as pd
# from fugue_sql import fsql
#Single model classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
#Ensemble classifications models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#Model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,confusion_matrix, classification_report, roc_auc_score,
f1_score, recall_score, precision_score, SCORERS)
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import numpy as np
import csv

In [ ]:
path='/home/o/oananbeh/notebook/src/TabTransformer-multiclass_discharge/'
Modelname = path+'RandomForestClassifier_model.sav'


# Functions

In [ ]:
def get_prob(test_features,classType):
    # load the model from disk
    model = pickle.load(open(Modelname, 'rb'))
    y_proba = model.predict_proba(test_features)
   
    second_proba = y_proba[:, classType]
    c=(np.mean(second_proba))
    return c

In [ ]:
def create_dict_rules1(list_rules):
    list_keys=[k.split(':')[0] for k in list_rules]
    list_values=[k.split(':')[-2]+':'+k.split(':')[-1] if ('>' in k or '<' in k) else k.split(':')[-1] for k in list_rules ]
    rule_dict = {list_keys[i]: list_values[i] for i in range(len(list_keys))}
    return rule_dict

In [ ]:
#Platelets:< 50000

In [ ]:
c=['ABGF_Pa_O2:0_80', 'CSA_Cough:1', 'Platelets:< :50000']
result=create_dict_rules1(c)
result

In [ ]:
def create_dict_rules(list_rules):
    list_keys=[k.split(':')[0] for k in list_rules]
    list_values=[k.split(':')[-2]+k.split(':')[-1] if ('>' in k or '<' in k) else k.split(':')[-1] for k in list_rules ]
    rule_dict = {list_keys[i]: list_values[i] for i in range(len(list_keys))}
    return rule_dict

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))*10


# ML

In [ ]:
Load_DF=pd.read_csv(path+'discharge_final_ML.csv')
Load_DF = Load_DF.astype('category')
 

In [ ]:
Load_DF.shape

# RandomForestClassifier

In [ ]:
df_train = Load_DF.drop('OUTCOME', axis=1)
train_labels = Load_DF['OUTCOME']

In [ ]:
df_train.shape[1]

In [ ]:
encoders = []
features = []
for i in range(89):
    encoder_i = OneHotEncoder(handle_unknown='ignore')
    i_column = df_train.columns[i]
    encoder_i.fit(df_train[i_column].values.reshape(-1, 1))
    encoded_feature = encoder_i.transform(df_train[i_column].values.reshape(-1, 1))
    encoded_feature = encoded_feature.toarray()
    features.append(encoded_feature)
    encoders.append(encoder_i)

def get_oh(df):
    features_test = []
    for i in range(89):
        encoder_i = encoders[i]
        i_column = Load_DF.columns[i]
        encoded_test = encoder_i.transform(df[i_column].values.reshape(-1, 1))
        encoded_test = encoded_test.toarray()
        features_test.append(encoded_test)
    return  np.concatenate(features_test, axis=1)

train_features = np.concatenate(features, axis=1)


In [ ]:
train_features.shape, train_labels.values.shape

# prob

In [ ]:
#Load the DataSet
association_results=path+'CSV_Association_results_Lab_discharge=<1_0.2_0.5_visit1.csv'
process_df=pd.read_csv(association_results)
classType=0 # o or 1 which i need to get the prob for it  
name='discharge=<1' # died or discharge or AllData

In [ ]:
#get all the rule with 0.7 Support
# process_df=process_df[(process_df['Support']>=0.5)]

In [ ]:
process_df

In [ ]:
process_df['Support'].count()

In [ ]:
# process_df.rename(columns={'G.AGE': 'AGE','ABGF.Ph':'ABGF_Ph',
# 'ABGF.Pa O2':'ABGF_Pa_O2','ABGF.Pa CO2':'ABGF_Pa_CO2','ABGF.HCO3-':'ABGF_HCO3',
# 'PEFF.Temperature':'PEFF_Temperature','PEFF.Respiratory Rate':'PEFF_Respiratory_Rate',
# 'PEFF.Pulse':'PEFF_Pulse','PEFF.BP_Systolic':'PEFF_BP_Systolic','PEFF.BP_Diastolic':'PEFF_BP_Diastolic',
# 'PEFF.Glasgow':'PEFF_Glasgow','D-DIMER':'D_DIMER','Pro-BNP':'Pro_BNP','Vitamin D':'Vitamin_D',
# 'C.Diabetes':'C_Diabetes','C.HTN':'C_HTN','CSA.Fever':'CSA_Fever','CSA.SOB':'CSA_SOB',
# 'CSA.Cough':'CSA_Cough','MPA.Antibiotics':'MPA_Antibiotics','MPA.Favipiravir':'MPA_Favipiravir',
# 'MPA.Dexamethasone':'MPA_Dexamethasone','MPA.Convalescent plasma transfusion':'MPA_Convalescent_plasma_transfusion',
# 'MPA.Clexan or Heparine':'MPA_Clexan_or_Heparine','MPA.Tocilizumab':'MPA_Tocilizumab'}, inplace=True)

In [ ]:
process_df['Rule']=process_df['Rule'].astype('string')

In [ ]:
process_df['Rule'] = process_df['Rule'].apply(eval)

In [ ]:
df_orginal=pd.read_csv(path+'SMOT_discharge_DataSet.csv')
# df_orginal=df_orginal.astype('object')

In [ ]:
import csv
with open(path+'prob_result_RandomForest_'+name+'.csv', 'w') as writeResult:
    CSV_writer = csv.writer(writeResult, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    CSV_writer.writerow(['Rule','prob_died'])
    for rule in process_df['Rule'] :
        # print(rule)
        rule_dict=create_dict_rules1(rule)
        df_query=''
        for k,v in rule_dict.items():  
            if k != list(rule_dict)[-1]:
                df_query=df_query+(k+"=='"+k+':'+str(v))+"' & " 
            else:
                df_query=df_query+(k+"=='"+k+':'+str(v))+"'"
       
        new_df=Load_DF.query(df_query)
        number_row=new_df.shape[0]
        if number_row==0:
             continue   
        # print(words,new_df.shape)
        # print(new_df)
        #get the onehot 
        test_features =get_oh(new_df)
        c=get_prob(test_features,classType)
        CSV_writer.writerow([rule,c])

In [ ]:
# import csv
# with open(path+'prob_result_'+name+'.csv', 'w') as writeResult:
#     CSV_writer = csv.writer(writeResult, delimiter=',', quoting=csv.QUOTE_MINIMAL)
#     CSV_writer.writerow(['Rule','prob_died'])
#     for rule in process_df['Rule'] :
#         # print(rule)
#         rule_dict=create_dict_rules(rule)
#         df_query=''
#         for k,v in rule_dict.items():   
#             if k != list(rule_dict)[-1]:
#                 df_query=df_query+(k+"=='"+k+':'+str(v))+"' & " 
#             else:
#                 df_query=df_query+(k+"=='"+k+':'+str(v))+"'"
       
#         new_df=Load_DF.query(df_query)
#         number_row=new_df.shape[0]
#         print(df_query,number_row)
#         if number_row==0:
#              continue   
#         # print(words,new_df.shape)
#         #get the onehot 
#         test_features =get_oh(new_df)
#         c=get_prob(test_features,classType)
#         CSV_writer.writerow([rule,c])

In [ ]:
df=pd.read_csv(path+'prob_result_RandomForest_'+name+'.csv')

In [ ]:
df.shape

In [ ]:
x=df['prob_died']


In [ ]:
np.min(x) ,np.max(x)

In [ ]:
#Normalize Data
scaled_x = NormalizeData(x)
df['scaled']=scaled_x


In [ ]:
df.to_csv(path+'prob_result_RandomForest_'+name+'.csv',index=False)

# Get the Orgenal Data /TabTransformer

In [ ]:
# x=['ABGF_Pa_O2:0_80', 'CSA_SOB:1', 'LDH:> :225']
# result=create_dict_rules(x)
# result

In [ ]:
from keras.models import load_model
tabtransformer_model = load_model(path+'my_model.h5')

In [ ]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
path='/home/o/oananbeh/notebook/src/TabTransformer-multiclass_discharge/'
dataSetPath=path+'SMOT_discharge_DataSet.csv'
data=pd.read_csv(dataSetPath)
CSV_HEADER=list(data.columns)

In [ ]:
NUMERIC_FEATURE_NAMES=[ 'AGE','PEFF_Temperature', 'PEFF_Respiratory_Rate', 'PEFF_Pulse',
       'PEFF_BP_Systolic', 'PEFF_BP_Diastolic', 'PEFF_Glasgow',
       'ABGF_Ph', 'ABGF_Pa_O2', 'ABGF_Pa_CO2','ABGF_HCO3',
       'WBC', 'PNN', 'Lymphocytes', 'Hemoglobin', 'Platelets',
       'Creatinine', 'ALT', 'LDH', 'FERRITIN', 'D_DIMER', 'CRP',
       'PROCALCITONI', 'TROPONIN', 'Pro_BNP', 'PTT', 'Vitamin_D', 'IL6',
       ]

In [ ]:
CATEGORICAL_FEATURES=['PatientId','G.Nationality','Gender','C_Diabetes', 'C_HTN',
       'C.Heart ischemic', 'C.Heart failure', 'C.Cardiomyopathies',
       'C.End stage renal', 'C.Hemodialysis', 'C.COPD',
       'C.Lung Interstitial Disease ', 'C.Bronchial Asthma',
       'C.Cerebrovascular', 'C.Neurologic (dementia)',
       'C.History of psychiatric', 'C.Cirrhosis', 'C.liver disease',
       'C.Obesity', 'C.Sick cell', 'C.Cancer', 'C.Solid organ transplant',
       'IS.Hematopoietic cell transplant', 'IS.HIV', 'IS.corticosteroids',
       'IS.Other immunosuppressing agents', 'IS.Other immunodeficiencies',
       'IS.Pregnancy', 'IS.Smoker',  'CSA_Fever', 'CSA_SOB', 'CSA.Chest pain', 'CSA.Confusion',
       'CSA.Hemoptysis', 'CSA.Diarrhea', 'CSA_Cough', 'CSA.Myalgia',
       'CSA.Headache', 'CSA.Abdominal Pain', 'CSA.Nausea or Vomiting',
       'CSA.Loss of Smell or Tast', 'PO.Condition', 'PO.Current Condition',
       'PEFF.Nasal Cannula', 'PEFF.Mask', 'PEFF.HFNC',
       'PEFF.If patient need prone position', 'PEFF.If patient intubated',
       'PEFF.If Patient required Psychiatric Consultation',
       'PEFF.Presence of thrombo-embolic', 'PEFF.Confirmed DVT',
       'PEFF.Confirmed Pulmonary embolism',
       'PEFF.Confirmed Myocardial infarction', 'PEFF.Confirmed CVA/TIA','Normal', 'Presence of consolidation',
       'Presence of ground glass opacities', 'Bilateral or unilateral',
       'Involve ≥ 50% the lung', 'Other']  

In [ ]:
BATCH_SIZE = 8
learning_rate = 0.001
weight_decay = 0.0001
 

In [ ]:
# # A list of column default values for each feature.
COLUMN_DEFAULTS = [
    [0.0] if feature_name in NUMERIC_FEATURE_NAMES  else ["NA"]
    for feature_name in CSV_HEADER
]
# The name of the target feature.
TARGET_FEATURE_NAME = 'OUTCOME'
# A list of the labels of the target features.
TARGET_LABELS = ['discharge=<1','discharge=<2','discharge=<3','discharge=<4','discharge>4']

 

In [ ]:

target_label_lookup = layers.StringLookup(
    vocabulary=TARGET_LABELS, mask_token=None, num_oov_indices=0
)

def prepare_example(features, target):
    # print(target)
    target_index = target_label_lookup(target)
    # weights = features.pop(WEIGHT_COLUMN_NAME), weights
    return features, target_index 

def get_dataset_from_csv(csv_file_path, batch_size=128, shuffle=False):
    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        column_defaults=COLUMN_DEFAULTS,
        label_name=TARGET_FEATURE_NAME,
        num_epochs=1,
        header=False,
        na_value="?",
        shuffle=shuffle,
    ).map(prepare_example, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False) 
    return dataset.cache()

In [ ]:
filename=path+'prob_result_TabTransformer'+name+'.csv'

with open(filename, 'a') as writeResult:
    CSV_writer = csv.writer(writeResult, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    CSV_writer.writerow(['discharge=<1','discharge=<2','discharge=<3','discharge=<4','discharge>4','Rule','Support','Confidence'])

In [ ]:
# import numpy

# def run_experiment1(
#     model,
#     test_data_file,
#     batch_size,
#     rule,
# ):

#     validation_dataset = get_dataset_from_csv(test_data_file, batch_size)
#     y_proba = model.predict(validation_dataset)  
#     df_prob=pd.DataFrame(columns=['discharge=<2','discharge=<4','died=<3','discharge>4','died=>3','discharge=<1',' discharge=<3'],data= y_proba)

#     print(df_prob)
#     result_argmax=(np.argmax(y_proba))
#     c=(np.mean(result_argmax))
#     with open(filename, 'a') as writeResult:
#         CSV_writer = csv.writer(writeResult, delimiter=',', quoting=csv.QUOTE_MINIMAL)
#         CSV_writer.writerow([rule,c])

In [ ]:
import numpy
def run_experiment1(
    model,
    test_data_file,
    batch_size,
    rule,
    Support,
    Confidence,
):

    validation_dataset = get_dataset_from_csv(test_data_file, batch_size)
    y_proba = model.predict(validation_dataset)  
    df_prob=pd.DataFrame(columns=['discharge=<1','discharge=<2','discharge=<3','discharge=<4','discharge>4'],data= y_proba)
    f=df_prob.mean()
    y=f.to_dict()
    y["rule"] = rule
    y['Support']=Support
    y['Confidence']=Confidence
    result_argmax=(np.argmax(y_proba))
    c=(np.mean(result_argmax))
    with open(filename, 'a') as writeResult:
        CSV_writer = csv.writer(writeResult, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        CSV_writer.writerow(y.values())

In [ ]:
#get the Orginal Data
allDataFrame=[]
count=1
for row in process_df.itertuples():
    rule=row.Rule   
    result=create_dict_rules(rule)
    df_query=''
    newQuery=''
    query=[]
    for k,v in result.items():    
        value=v.split("_")
        if (len(value)>1):
            f=value[0]
            s=value[1]
            query.append(f+'<='+k + '<='+s)
        else:
            if ('>' in v or '<' in v):
                query.append(k+v[0]+v[1:])
            else:
                query.append(k+"=="+v)
    for q in query:
        if q != query[-1]:
         newQuery=newQuery+''.join(q) +" & "
        else:
            newQuery=newQuery+''.join(q)
    rule_df=df_orginal.query(newQuery)
    number_row=rule_df.shape[0]
    if number_row==0:continue
    for feature in CATEGORICAL_FEATURES[2:]:
        rule_df[feature] = data[feature].map({1:'yes', 0:'no'})
    # rule_df[CATEGORICAL_FEATURES]=rule_df[CATEGORICAL_FEATURES].astype('object')
    #Split thr Data into training and testing
    rule_df.to_csv(path+'rule_output/'+str(count)+'.csv', index=False, header=False)
    testingDdataSet=path+'rule_output/'+str(count)+'.csv'
    history = run_experiment1(
    model=tabtransformer_model,
    test_data_file=testingDdataSet,
    batch_size=BATCH_SIZE,
    rule=rule,
    Support= row.Support  ,
    Confidence=row.Confidence,)
    count+=1

In [ ]:
df=pd.read_csv(path+'prob_result_TabTransformer'+name+'.csv')

In [ ]:
x=df['prob_died'].astype('float')

In [ ]:
x

In [ ]:
#Normalize Data
scaled_x = NormalizeData(x)
df['scaled']=scaled_x

In [ ]:
scaled_x

In [ ]:
df.to_csv(path+'prob_result_TabTransformer'+name+'.csv',index=False)